# Plotting N-D data

Data with any number of dimensions can also be plotted in Scipp.

In [ ]:
import numpy as np
import scipp as sc
from scipp.plot import plot

## Default representation

Data with 3 or more dimensions are by default represented by a 2-D image, accompanied by sliders to navigate the extra dimensions (one slider per dimension above 2).

In [ ]:
N = 50
M = 40
L = 30
K = 20
xx = np.arange(N, dtype=np.float64)
yy = np.arange(M, dtype=np.float64)
zz = np.arange(L, dtype=np.float64)
qq = np.arange(K, dtype=np.float64)
x, y, z, q = np.meshgrid(xx, yy, zz, qq, indexing='ij')
b = N/20.0
c = M/2.0
d = L/2.0
r = np.sqrt(((x-c)/b)**2 + ((y-c)/b)**2 + ((z-d)/b)**2  + ((q-d)/b)**2)
a = np.sin(r)
d = sc.Dataset()
d.coords['x'] = sc.Variable(['x'], values=xx)
d.coords['y'] = sc.Variable(['y'], values=yy)
d.coords['z'] = sc.Variable(['z'], values=zz)
d.coords['Q_x'] = sc.Variable(['Q_x'], values=qq)
d['Some4Ddata'] = sc.Variable(['x', 'y', 'z', 'Q_x'], values=a)
plot(d)

### Slider controls

- Each dimension comes with two sliders to control the position of the slice and its thickness.

- Upon figure creation, the thickness is set to cover the entire range of the additional dimension, and the data is summed along these dimensions.

- Reducing the slice thickness will also change the color range, and the `Rescale` button can be used to automatically rescale the colorbar to the limits of the currently displayed data.

### Changing axes dimensions

By default, the two innermost dimensions are used for the image, and the rest will be allocated to a slider.
This can be changed, either interactively using the buttons, or by specifying the order of the axes in the `plot` command:

In [ ]:
plot(d, axes={'x': 'x', 'y': 'Q_x'})

### Profile picking

Finally, each dimension also comes with a `Profile` button which allows to display one of the additional dimensions as a profile underneath the main plot.

- When hovering the mouse over the top image, the profile below is updated according to the mouse position.
- Clicking on the image will save the current profile with a random color.
- Clicking on an existing marker on the image will delete the corresponding saved profile.

## 3D visualization

<div class="alert alert-info">

**Note**

3D visualization requires `pythreejs` to be installed. Use either `pip` or `conda`:
```
- conda install -c conda-forge pythreejs
- pip install pythreejs
```

</div>

It is also possible to use a 3d projection with `projection="3d"`.
If the data is a dense 3(+)D data array, a mesh of points filling the space will be created:

In [ ]:
plot(d, projection='3d')

You can adjust opacities and create a cut surface to slice your data in 3D using the control buttons below the scene. When using a cut surface, the upper value of the opacity slider controls the opacity of the slice, while the lower value of the slider controls the opacity of the background.

The 3D projection will also accept a dataset that contains position vectors. The way to use these vectors as positions to generate a 3D scatter plot is to use the `positions` keyword argument:

In [ ]:
N = 1000
M = 100
theta = np.random.random(N) * np.pi
phi = np.random.random(N) * 2.0 * np.pi
r = 10.0 + (np.random.random(N) - 0.5)
x = r * np.sin(theta) * np.sin(phi)
y = r * np.sin(theta) * np.cos(phi)
z = r * np.cos(theta)

tof = np.arange(M).astype(np.float)
a = np.arange(M*N).reshape([M, N]) * np.sin(y)
d2 = sc.Dataset()
d2.coords['xyz'] = sc.Variable(['xyz'], values=np.array([x, y, z]).T,
                               dtype=sc.dtype.vector_3_float64)
d2.coords['tof'] = sc.Variable(['tof'], values=tof)
d2['a'] = sc.Variable(['tof', 'xyz'], values=a)
plot(d2, projection="3d", positions="xyz")

## LAMP's Superplot
A `1d` projection is also available for multi-dimensional data, with the possibility to keep/remove lines that are plotted, a behaviour we copied from LAMP's [Superplot](https://github.com/mantidproject/documents/blob/master/Requirements/Visualisation_and_Analysis/superplot.md) which was very popular in the neutron physics community.

In [ ]:
plot(d, projection='1d')

## Convenience Methods

A small number of convenience methods are included in the `plot` module which provide shortcuts to the different projections.
These are `superplot` (=`projection="1d"`), `image` (=`projection="2d"`), and `scatter3d` (=`projection="3d"`), and they are used in the following way:

In [ ]:
from scipp import plot as pl
pl.superplot(d)